In [8]:
#%save Executeur_receveur.py 5

In [5]:
import os, sys, time, random
import subprocess


path = "../outils"
if path not in sys.path : 
    sys.path.append (path)
    
from Gestion_commandes import Gestion_commandes, Gestion_reponses, Gestion_erreurs
from lire_dico_json import lire_dico_json

path = "../Alimentation"
if path not in sys.path : 
    sys.path.append (path)
from Alimentation import Alimentation, Mise_a_jour_alimentation

path = "../Apprentissage"
if path not in sys.path : 
    sys.path.append (path)
from Apprentissage import Apprentissage, Mise_a_jour_apprentissage


from Execution import Execution,  Mise_a_jour_execution
from Gestion_etat import Gestion_receveur

    
    
class test1 () :
    
    def __init__ (self, arg) :
        
        self.arg = arg
        
    def run ( self,) :
        
        texte = self.arg ['test']
        print ('recu =', texte)
        self.arg ['envoie'] = 'à vous le test'
        time.sleep (1) 
        
        
        return "merci de fonctionner"
    
class test2 () :
    
    def __init__ (self, arg) :
        
        self.arg = arg
        
    def run ( self,) :
        
        print ('test2 crash')
        
        1/0
        
        
class test3 () :
    
    def __init__ (self, arg) :
        
        self.arg = arg
        
    def run ( self,) :
        return "coucou"
        
    

    
    
            
    
    
    
    

class Executeur () :
    
    def __init__ (self, nom_worker) :
        
        self.nom_worker = nom_worker
        
        self.Gestion_commandes = Gestion_commandes ()
        self.Gestion_reponses = Gestion_reponses()
        
        
           
        
        
        self.path = '../data/general/parametres/'
        pathExecuteur = self.path + 'dico_executeur.json'
        dico_executeur = lire_dico_json (pathExecuteur)
        self.temps_attente = dico_executeur ['temps_attente']
        
        
        
        self.dico_execution = {'test1' : test1,
                               'test2' : test2,
                               'test3' : test3,
                               'Alimentation' : Alimentation ,
                               'Apprentissage' : Apprentissage,
                               'Execution' : Execution
                              }
        
        self.isStop = False
        
        
            
        
        
         
        
        
    def run (self,) :
              
        while (True) :
            
            try:
                if self.isStop :
                    break
                    
                message = self.Gestion_commandes.get_message ()
                
                if message is None :
                    
                    temps = random.uniform(1, self.temps_attente + 1) - 1.0
                    time.sleep (temps)
                    continue
                
                
                
                # retour running sur ce worker critique (worker crash)
                message_retour = message
                message_retour ['nom_worker'] = self.nom_worker
                message_retour ['etat'] = 'RUNNING'
                       
                self.Gestion_reponses.put_message (message_retour,)
                
                nom_appel = message ['nom_appel']
                class_appel = self.dico_execution [nom_appel]
                arg = message ['arg']
                resultat = class_appel(arg).run()
                
                # retour resultat 
                message_retour = message
                message_retour ['resultat'] = resultat
                message_retour ['etat'] = 'OK'
                
                self.Gestion_reponses.put_message (message_retour,)
                
            except Exception as e:
                message_retour = {}
                message_retour ['etat'] = 'KO'
                message_retour ['exception'] = str(e)
                self.Gestion_reponses.put_message (message_retour,)
                
                
            continue
            
    def lancement_executeur (self,) :
        self.creation_lancement ()
        
        bashCommand = "python lancement_executeur.py"
        self.process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        return
    
    def arret_executeur (self,) :
        self.process.terminate ()
        return
    
    def creation_lancement (self,):
        worker = str(self.worker)
        data = """from Executeur_receveur import Executeur
Executeur ("""+worker+""").run()
"""
        
        path_file = "lancement_executeur.py"
        f = open (path_file, 'w')
        f.write (data)
        f.close()
            
class Receveur (Gestion_receveur) :
    
    def __init__ (self,) :
        
        
        
        super().__init__ ()
        
        self.Gestion_reponses = Gestion_reponses()
        
        
        self.path = '../data/general/parametres/'
        pathExecuteur = self.path + 'dico_receveur.json'
        dico_executeur = lire_dico_json (pathExecuteur)
        self.temps_attente = dico_executeur ['temps_attente']
        
        
        self.dico_execution = {'test1' : test1,
                               'test2' : test2,
                               'test3' : test3,
                               'Alimentation' : Mise_a_jour_alimentation ,
                               'Apprentissage' : Mise_a_jour_apprentissage,
                               'Execution' : Mise_a_jour_execution
                              }
           
        self.isStop = False
        
        
            
        
        
         
        
        
    def run (self,) :
        
        
        
        while (True) :
            
            try:
                if self.isStop :
                    break
                message = self.Gestion_reponses.get_message ()
                
                if message is None :
                    
                    temps = random.uniform(1, self.temps_attente + 1) - 1.0
                    time.sleep (temps)
                    continue
                              
                
                etat = essage ['etat']
                if etat == 'RUNNING' :
                    self.tache_running (message)
                    continue
                elif etat == 'OK' :
                    # Mise a jour evenement
                    ##########################
                    self.tache_OK (message)
                    continue
                elif etat == 'KO' :
                    self.tache_KO (message)
                    continue
                raise ValueError
                
            except Exception as e:
                message_retour = {}
                message_retour ['etat'] = 'erreur_receveur'
                message_retour ['exception'] = str(e)
                self.Gestion_erreurs.put_message (message_retour,)
                
            continue
            
    def lancement_receveur (self,) :
        self.creation_lancement ()
        
        bashCommand = "python lancement_receveur.py"
        self.process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        return
    
    def arret_receveur (self,) :
        self.process.terminate ()
        return
    
    def creation_lancement (self,):
        
        data = """from Executeur_receveur import Receveur
Receveur ().run()
"""
        
        path_file = "lancement_receveur.py"
        f = open (path_file, 'w')
        f.write (data)
        f.close()
        return
    

                
                   

    
          
            


In [4]:
#%save test_executeur.py 2

In [3]:
import sys,  time, os, json
from datetime import datetime
import subprocess
from pprint import PrettyPrinter 
def PP (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)

from Executeur import Executeur

path = "../outils"
if path not in sys.path : 
    sys.path.append (path)
    
from Gestion_commandes import Gestion_commandes
from Gestion_reponses import  Gestion_reponses

from lire_dico_json import lire_dico_json

def test_executeur () :


    def clean () :
        def detruire (pathFile,) :
            if os.path.exists(pathFile) :
                os.remove(pathFile)
                print ('fichier detruit en ', pathFile)
            else: 
                print ('n existe pas = ', pathFile)

        def verif_vide (pathFile) :
            try:
                f = open (pathFile, 'r')
                data = f.read ()
                resultat = json.loads (data)
                assert resultat == {} , "fichier " + pathFile + " non vide ?"
            except :
                assert False, "fichier " + pathFile + " n'existe pas ?"
            return


        # initialisation à zero des messages    
        nom_environnement = 'general'

        path = '../data/'+ nom_environnement + '/parametres/'
        pathReponses = path + 'dico_reponses.json'
        dico_reponses = lire_dico_json (pathReponses)

        pathFile = path + dico_reponses ['pathFile']
        pathFile_lock = pathFile + ".lock"

        detruire (pathFile_lock)
        detruire (pathFile,)



        pathCommandes = path + 'dico_commandes.json'
        dico_commandes = lire_dico_json (pathCommandes)
        pathFile = path + dico_commandes ['pathFile']
        pathFile_lock = pathFile + ".lock"
        detruire (pathFile_lock,)
        detruire (pathFile,)

        # test init on test en réel le demarrage avec mise à vide de la liste des messages

        E = Executeur()




        path = '../data/'+ nom_environnement + '/parametres/'
        pathReponses = path + 'dico_reponses.json'
        dico_reponses = lire_dico_json (pathReponses)

        pathFile = path + dico_reponses ['pathFile']
        verif_vide (pathFile)

        pathCommandes = path + 'dico_commandes.json'
        dico_commandes = lire_dico_json (pathCommandes)
        pathFile = path + dico_commandes ['pathFile']
        verif_vide (pathFile)

        return

    clean()
    
    
    
    
    
    G = Gestion_commandes ()
    R = Gestion_reponses ()

    date = datetime.now()
    date_voulu = date. strftime('%y/%m/%d %H:%M:%S.%f')
    #assert isinstance(date_voulu, type(' '))

    message = {'nb_max_worker' : 1, 'nb_max_erreur' : 1, 'date_voulu' : date_voulu, 'nom_appel' : 'test1',
           'parametres' : {'arg' : {'test' : 'hello world nouveau'}, 'nom_appel' : 'test1'}}

    dico, numero_commande = G.put_new_message (message)

    #assert numero_commande == 'commande_0'

    time.sleep (2)

    dico, resultat = R.get_dico_reponses_done ()
    
    print ('resultat test1 =', resultat)

    voulu = {'commande_0': {'resultat': 'merci de fonctionner',
                          'parametres': {'nom_appel': 'test1',
                                         'arg': {'test': 'hello world nouveau',
                                                 'envoie': 'à vous le test'},
                                        },
                          'etat': 'non_lu'},
          }
    
    
    
    #assert resultat == voulu


    message = {'resultat': 'merci',
              'parametres': {'nom_appel': 'test2',
                          'arg': {'test': 'hello world nouveau',
                                  'envoie': 'à vous le test'},
                          },
                'etat': 'done'}



    print ('envoi test2')
    dico, numero_commande = G.put_new_message (message)

    #assert numero_commande == 'commande_1'

    time.sleep (1.5)

    dico, resultat = R.get_dico_reponses_crash ()
    print ('resultat crash test2 =', resultat)
    nom_environnement = 'test'   
    arg = {}
    arg ['nom_environnement'] = nom_environnement
    arg ['isPurge_existing_index'] = True
    arg ['nom_fichier'] =  'evenements.txt'
    
    message = {'parametres' : {'arg' : arg, 'nom_appel' : 'Alimentation'}}
    
    print ('demande pour alimentation')
    dico, numero_commande = G.put_new_message (message)
    time.sleep (2)
    
    dico, resultat = R.get_dico_reponses_done ()
    print ('resultat alimentation =', resultat)
    
    
    # test Apprentissage
    nom_environnement = 'test'
    arg = {}
    arg ['isTrace'] = False
    nom_environnement = 'test'
    arg ['nom_environnement'] = nom_environnement
    
    arg ['variable_min'] = '2021-02-01 00:00:00.000000'
    arg ['variable_max'] = '2021-02-09 00:00:00.000000'
    
    message = {'parametres' : {'arg' : arg, 'nom_appel' : 'Apprentissage'}}
    print ('demande pour Apprentissage')
    dico, numero_commande = G.put_new_message (message)
    
    time.sleep (2)
    
    dico, resultat = R.get_dico_reponses_done ()
    print ('resultat apprentissage =', resultat)
    
    arg = {}
    arg ['variable_min'] = '2021-02-01 00:00:00.000000'
    arg ['variable_max'] = '2021-02-09 00:00:00.000000'
    arg ['pas'] = 'semaine'
    arg ['nom_environnement'] = 'test'
    
    arg ['workers'] = 6
    arg ['local_scheduler'] = True
    
    message = {'parametres' : {'arg' : arg, 'nom_appel' : 'Execution'}}
    print ('demande pour Execution')
    dico, numero_commande = G.put_new_message (message)
    
    time.sleep (2)
    
    dico, resultat = R.get_dico_reponses_done ()
    print ('resultat Execution =', resultat)
    

    

    
if __name__ == '__main__' ;
   
    test_executeur () 
    
        
    
        
        
    print ('fin test_executeur')
    
    

fichier detruit en  ../data/general/parametres/echanges/reponses.json.lock
fichier detruit en  ../data/general/parametres/echanges/reponses.json
fichier detruit en  ../data/general/parametres/echanges/commandes.json.lock
fichier detruit en  ../data/general/parametres/echanges/commandes.json
resultat test1 = {'commande_0': {'resultat': 'merci de fonctionner', 'parametres': {'nom_appel': 'test1'}, 'etat': 'non_lu'}}
envoi test2
resultat crash test2 = {'commande_1': {'etat': 'non_lu', 'exception': 'division by zero'}}
demande pour alimentation
resultat alimentation = {'commande_2': {'resultat': None, 'parametres': {'nom_appel': 'Alimentation'}, 'etat': 'non_lu'}}
demande pour Apprentissage
resultat apprentissage = {'commande_3': {'resultat': None, 'parametres': {'nom_appel': 'Apprentissage'}, 'etat': 'non_lu'}}
demande pour Execution
resultat Execution = {'commande_4': {'resultat': '../data/test/data/semaine/2021-02-09_00.00.00.000000/', 'parametres': {'nom_appel': 'Execution'}, 'etat': '

In [4]:
from pprint import PrettyPrinter 
def P (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)


def lire_file (pathFile) :
    f = open (pathFile, 'r')
    data = f.read()
    return json.loads (data)

pathEvenements = '../data/test/parametres/dico_evenements_2.json'
pathSysteme = '../data/test/parametres/dico_systeme_2.json'
nom_environnement = 'test'

dico_evenements = lire_file (pathEvenements)
dico_systeme = lire_file (pathSysteme)

print ('dico_evenements:')
P(dico_evenements)


dico_evenements:
{   'alimentation': {   'execution': {   'date_debut_alimentation': '2021-02-01 '
                                                                    '00:00:00',
                                         'dico_ajout': {   'ajout_0000000000': {   'date_execution': '2021-05-10 '
                                                                                                     '19:44:32.348454',
                                                                                   'date_max': '2021-02-08 '
                                                                                               '11:41:19.754879',
                                                                                   'date_min': '2021-02-01 '
                                                                                               '00:57:06.244944',
                                                                                   'nombre_erreur': 0,
                                     